In [ ]:
# Optional: setup NoTexBook theme
%load_ext notexbook

%texify

# Model Inversion Attack

In [ ]:
import torch as th
import numpy as np

from matplotlib import pyplot as plt

%matplotlib inline

In [ ]:
# Tweak to reuse the Python modules defined in previous section
import sys, os
from pathlib import Path

sys.path.insert(0, os.path.join(os.path.abspath(os.path.curdir), "..", "2-ml-models-attacks"))

In [ ]:
from dataset import ORLFaces
from torchvision.transforms import ToTensor

In [ ]:
# NOTE: This is a hack to get around "User-agent" limitations when downloading MNIST datasets
#       see, https://github.com/pytorch/vision/issues/3497 for more information
from six.moves import urllib

opener = urllib.request.build_opener()
opener.addheaders = [("User-agent", "Mozilla/5.0")]
urllib.request.install_opener(opener)

from pathlib import Path
import os

DATA_FOLDER = Path(os.path.join(os.path.abspath(os.path.curdir), "..")) / "data"

In [ ]:
orl_faces_train = ORLFaces(
    root=DATA_FOLDER, download=True, split="train", transform=ToTensor()
)
orl_faces_test = ORLFaces(root=DATA_FOLDER, download=True, split="test", transform=ToTensor())

In [ ]:
orl_faces_train.data.shape, orl_faces_test.data.shape

In [ ]:
from torch.utils.data import DataLoader

train_loader = DataLoader(
    orl_faces_train, batch_size=32, shuffle=False, drop_last=False
)

In [ ]:
# Reconstruction Attack Settings
# See Paper, Section 5.2 - Reconstruction Attack
α = 5000
β = 100
γ = 0.99
λ = 0.1

In [ ]:
from models import SoftmaxRegression, MLP

In [ ]:
from pathlib import Path

CHECKPOINT_FOLDER = Path("./checkpoints/")


def load_weights(model, model_name: str = None) -> th.TensorType:
    if model_name is None or not model_name:
        model_name = model.__class__.__name__.lower()
    w_file = CHECKPOINT_FOLDER / f"{model_name}.pt"
    try:
        weights = th.load(open(w_file, "rb"))
    except FileNotFoundError:
        print(f"Model Weights file {w_file} does not exist! Please check.")
        return None
    return weights

In [ ]:
softmax_reg = SoftmaxRegression()
weights = load_weights(softmax_reg, model_name="softmax_reg_opacus_test")

weights["regression.weight"] = weights["_module.regression.weight"]
_ = weights.pop("_module.regression.weight")

weights["regression.bias"] = weights["_module.regression.bias"]
_ = weights.pop("_module.regression.bias")

if weights is not None:
    softmax_reg.load_state_dict(weights)

In [ ]:
def process(im_flatten):
    max_v = th.max(im_flatten)
    min_v = th.min(im_flatten)
    return (im_flatten - min_v) / (max_v - min_v)

In [ ]:
def mi_face(model, target_label):
    aim_tensor = th.zeros(1, 112 * 92)
    aim_tensor.requires_grad = True

    lossn_1 = 10
    b = 0
    g = 0

    out = model(aim_tensor.detach())
    _, pred = th.max(out, 1)
    print(pred)
    print(f"original input image {target_label}")
    plt.imshow(
        np.transpose(aim_tensor.detach().reshape(1, 112, 92).numpy(), (1, 2, 0)),
        cmap="Greys",
    )
    plt.show()
    print(
        f"original input image predict label {target_label} - predict label: {pred.item()}"
    )

    criterion = th.nn.NLLLoss()

    for i in range(α):
        out = model(aim_tensor)
        if aim_tensor.grad is not None:
            aim_tensor.grad.zero_()
        out = out.reshape(1, 40)
        target_class = th.tensor([target_label])
        loss = criterion(out, target_class)
        loss.backward()
        aim_grad = aim_tensor.grad

        # SGD Step
        # see https://pytorch.org/docs/stable/generated/torch.optim.SGD.html#torch.optim.SGD
        aim_tensor = aim_tensor - (λ * aim_grad)
        aim_tensor = process(aim_tensor)
        aim_tensor = th.clamp(aim_tensor.detach(), 0, 1)
        aim_tensor.requires_grad = True
        if loss >= lossn_1:
            b += 1
            if b > β:
                break
        else:
            b = 0
        lossn_1 = loss
        if loss < γ:
            break

    print(f"Attack completed at {i} iterations")
    out = model(aim_tensor.detach())
    _, pred = th.max(out, 1)
    print(pred)
    print(f"inverted image {target_label}")
    plt.imshow(
        np.transpose(aim_tensor.detach().reshape(1, 112, 92).numpy() * 255, (1, 2, 0)),
        cmap="Greys",
    )
    plt.show()

In [ ]:
for cl in range(10):
    mi_face(softmax_reg, cl)